In [158]:
from jaqs_fxdayu.data import DataView
from jaqs_fxdayu.data.dataservice import LocalDataService
import pandas as pd
import talib as ta
import warnings
warnings.filterwarnings("ignore")

start = 20150101
end = 20180101

factor = ['grossmargin','ROE','ROE5','ROA','ROA5','EPS','DEGM','CTOP','CTP5','ETOP','ETP5','PB','PCF','PE','PS','bps','currency','volume']
check_factor = ','.join(factor)

dv_props = {'start_date': start, 'end_date': end, 'universe':'000001.SH',
         'fields': check_factor,
         'freq': 1,
         "prepare_fields": True}
    


ds = LocalDataService(fp=r'/Users/xiaomuluo/Win7/E/kuaipan/公开课/July/量化OFO/作业2/data')
dv = DataView()
dv.init_from_config(dv_props, data_api=ds)
dv.prepare_data()

Initialize config success.
Query data...
Query data - query...
NOTE: price adjust method is [post adjust]
当前请求daily...
{'adjust_mode': None, 'fields': 'trade_date,trade_status,symbol,low_adj,volume,close_adj,high_adj,open,vwap,high,low,close,open_adj,vwap_adj'}
下载进度647/1388.
下载进度1294/1388.
当前请求daily...
{'adjust_mode': 'post', 'fields': 'open,vwap,high,low,close,symbol,trade_date'}
下载进度647/1388.
下载进度1294/1388.
当前请求query...
{'fields': 'PS,PB,ROE,ETP5,ROA,EPS,ETOP,PCF,DEGM,PE,ROA5,ROE5,CTP5,CTOP', 'view': 'factor'}
下载进度647/1388.
下载进度1294/1388.
    At fields 600005.SH, 600656.SH, 600832.SH, 601299.SH, 601313.SH
Query data - daily fields prepared.
    At fields 
Query data - quarterly fields prepared.
Query instrument info...
Query adj_factor...
Query benchmark...
Query benchmar member info...
Data has been successfully prepared.


In [2]:
dv.fields

['PS',
 'trade_status',
 'ETP5',
 'ROA',
 'high_adj',
 'EPS',
 'PCF',
 'bps',
 'ROA5',
 'ROE5',
 'close',
 'vwap_adj',
 'ann_date',
 'currency',
 'ETOP',
 'vwap',
 'DEGM',
 'quarter',
 'PE',
 'CTP5',
 'CTOP',
 'low_adj',
 'close_adj',
 'open',
 'high',
 'low',
 'PB',
 'index_weight',
 'ROE',
 'index_member',
 'grossmargin',
 'volume',
 'open_adj']

# 作业

要编写的因子  
'alpha14'  
'Tobt'  
'alpha156'  
'CashRateOfSalesLatest'  
'OperCashFlowPS'  
'alpha45'  
'alpha152'   
'alpha71'

In [157]:
#factor_list = ['alpha14', 'alpha45', 'alpha71', 'alpha152', 'alpha156', 'CashRateOfSalesLatest', 'OperCashFlowPS']
factor_list = ['OperCashFlowPS']

#--------------------------------------------------------- 
#test output
def test(factor,data):
    if not isinstance(data, pd.core.frame.DataFrame):
        raise TypeError('On factor {} ,output must be a pandas.DataFrame!'.format(factor))
    else:
        try:
            index_name = data.index.names[0]
            columns_name = data.columns.names[0]
        except:
            if not (index_name in ['trade_date','report_date'] and columns_name == 'symbol'):
                raise NameError('''Error index name,index name must in ["trade_date","report_date"],columns name must be "symbol" ''')
                
        index_dtype = data.index.dtype_str
        columns_dtype = data.columns.dtype_str
        
        if columns_dtype not in ['object','str']:
            raise TypeError('error columns type')
            
        if index_dtype not in ['int32','int64','int']:
            raise TypeError('error index type')


test_factor = True

if test_factor:   
    for factor in factor_list:
        data = globals()[factor]()
        test(factor,data)

Field name [net_cash_flows_oper_act] already exists.
Field name [capital_stk] already exists.


# alpha14

CLOSE-DELAY(CLOSE,5)

In [37]:
def alpha14():
    alpha14 = dv.add_formula('alpha14', 'close-Delay(close,5)', is_quarterly=False, add_data=True)
    return alpha14

# alpha45

(RANK(DELTA((((CLOSE \* 0.6) + (OPEN \* 0.4))), 1)) * RANK(CORR(VWAP, MEAN(VOLUME,150), 15)))

In [ ]:
(RANK(DELTA((((CLOSE \* 0.6) + (OPEN \* 0.4))), 1)) * RANK(CORR(VWAP, MEAN(VOLUME,150), 15)))

In [46]:
def alpha45():
    alpha45 = dv.add_formula('alpha45', 'Rank(Delta(((close*0.6)+(open*0.4)),1)) *  Rank(Correlation(vwap, Ts_Mean(volume, 150), 15))', is_quarterly=False, add_data=True)
    return alpha45

# alpha71

(CLOSE-MEAN(CLOSE,24))/MEAN(CLOSE,24)\*100

In [ ]:
(CLOSE-MEAN(CLOSE,24))/MEAN(CLOSE,24)*100

In [59]:
def alpha71():
    alpha71 = dv.add_formula('alpha71', '(close-Ts_Mean(close, 24))/Ts_Mean(close, 24)*100', is_quarterly=False, add_data=True)
    return alpha71

# alpha152

SMA(MEAN(DELAY(SMA(DELAY(CLOSE/DELAY(CLOSE,9),1),9,1),1),12)-MEAN(DELAY(SMA(DELAY(CLOSE/DELAY(CLOSE,9),1),9,1),1),26),9,1)

In [ ]:
SMA(MEAN(DELAY(SMA(DELAY(CLOSE/DELAY(CLOSE,9),1),9,1),1),12)-MEAN(DELAY(SMA(DELAY(CLOSE/DELAY(CLOSE,9),1),9,1),1),26),9,1)

In [78]:
# 定义指数平均计算函数-传入一个时间为索引,股票为columns的Dataframe,计算其指数平均序列
# SMAtoday=m/n * Pricetoday + ( n-m )/n * SMAyesterday;
def sma(df, n, m):
    a = n / m - 1
    r = df.ewm(com=a, axis=0, adjust=False)
    return r.mean()

def alpha152():    
    alpha152 = dv.add_formula("alpha152","SMA(Ts_Mean(Delay(SMA(Delay(close/Delay(close,9),1),9,1),1),12)-Ts_Mean(Delay(SMA(Delay(close/Delay(close,9),1),9,1),1),26),9,1)", is_quarterly=False, add_data=True, register_funcs={"SMA":sma})
    return alpha152

In [73]:
df = alpha71()
help(df.ewm)

Field [alpha71] is overwritten.
Help on method ewm in module pandas.core.generic:

ewm(com=None, span=None, halflife=None, alpha=None, min_periods=0, freq=None, adjust=True, ignore_na=False, axis=0) method of pandas.core.frame.DataFrame instance
    Provides exponential weighted functions
    
    .. versionadded:: 0.18.0
    
    Parameters
    ----------
    com : float, optional
        Specify decay in terms of center of mass,
        :math:`\alpha = 1 / (1 + com),\text{ for } com \geq 0`
    span : float, optional
        Specify decay in terms of span,
        :math:`\alpha = 2 / (span + 1),\text{ for } span \geq 1`
    halflife : float, optional
        Specify decay in terms of half-life,
        :math:`\alpha = 1 - exp(log(0.5) / halflife),\text{ for } halflife > 0`
    alpha : float, optional
        Specify smoothing factor :math:`\alpha` directly,
        :math:`0 < \alpha \leq 1`
    
        .. versionadded:: 0.18.0
    
    min_periods : int, default 0
        Minimum nu

# alpha156

(MAX(RANK(DECAYLINEAR(DELTA(VWAP, 5), 3)), RANK(DECAYLINEAR(((DELTA(((OPEN \* 0.15) + (LOW \*0.85)),2) / ((OPEN \* 0.15) + (LOW \* 0.85))) \* -1), 3))) \* -1)

In [84]:
def alpha156(): 
#(MAX(RANK(DECAYLINEAR(DELTA(VWAP, 5), 3)), RANK(DECAYLINEAR(((DELTA(((OPEN * 0.15) + (LOW *0.85)),2) / ((OPEN * 0.15) + (LOW * 0.85))) * -1), 3))) * -1)   
    alpha156 = dv.add_formula("alpha156","(Max(Rank(Decay_linear(Delta(vwap, 5), 3)), Rank(Decay_linear(((Delta(((open * 0.15) + (low *0.85)),2) / ((open * 0.15) + (low * 0.85))) * -1), 3))) * -1)", is_quarterly=False, add_data=True)
    return alpha156

# Tobt

收益与风险类  
type: float  
超额流动（Liquidity-turnover beta）。属于收益和风险类因子。数据更新时间：T日17点00分。  
P128

In [103]:
dv.add_field('cap_mrgn_paid', data_api=ds)

Field name [cap_mrgn_paid] already exists.


False

In [103]:
dv.add_field('TOBT')

Query data - query...
当前请求query...
{'fields': 'TOBT', 'view': 'factor'}
下载进度647/1388.
下载进度1294/1388.
    At fields 600005.SH, 600656.SH, 600832.SH, 601299.SH, 601313.SH
Query data - daily fields prepared.


True

In [104]:
dv.get_ts('TOBT')

symbol,600000.SH,600004.SH,600005.SH,600006.SH,600007.SH,600008.SH,600009.SH,600010.SH,600011.SH,600012.SH,...,603987.SH,603988.SH,603989.SH,603990.SH,603991.SH,603993.SH,603996.SH,603997.SH,603998.SH,603999.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20150105,1.0149,1.4196,NaN,1.3979,3.3887,0.8507,0.6123,0.7069,3.8325,1.6640,...,NaN,NaN,NaN,NaN,NaN,0.0788,NaN,NaN,NaN,NaN
20150106,0.9953,1.4191,NaN,1.3825,3.3870,0.8801,0.6110,0.7066,3.8267,1.6877,...,NaN,NaN,NaN,NaN,NaN,0.0786,NaN,NaN,NaN,NaN
20150107,0.9914,1.4214,NaN,1.3827,3.3977,0.8798,0.6086,0.7063,3.7954,1.6881,...,NaN,NaN,NaN,NaN,NaN,0.0789,NaN,NaN,NaN,NaN
20150108,0.9944,1.4200,NaN,1.3818,3.3892,0.8781,0.6070,0.7069,3.8265,1.6830,...,NaN,NaN,NaN,NaN,NaN,0.0792,NaN,NaN,NaN,NaN
20150109,0.9798,1.4178,NaN,1.3824,3.3913,0.8781,0.6034,0.7068,3.8203,1.6852,...,NaN,NaN,NaN,NaN,NaN,0.0801,NaN,NaN,NaN,NaN
20150112,0.9747,1.4177,NaN,1.3823,3.3965,0.8752,0.6042,0.7027,3.8340,1.6814,...,NaN,NaN,NaN,NaN,NaN,0.0804,NaN,NaN,NaN,NaN
20150113,0.9751,1.4180,NaN,1.3822,3.3901,0.8779,0.6045,0.7022,3.8396,1.6802,...,NaN,NaN,NaN,NaN,NaN,0.0802,NaN,NaN,NaN,NaN
20150114,0.9730,1.4153,NaN,1.3837,3.3900,0.8782,0.6056,0.7048,3.8329,1.6754,...,NaN,NaN,NaN,NaN,NaN,0.0792,NaN,NaN,NaN,NaN
20150115,0.9744,1.4181,NaN,1.3809,3.3921,0.8772,0.6059,0.7050,3.8401,1.6736,...,NaN,NaN,NaN,NaN,NaN,0.0788,NaN,NaN,NaN,NaN


In [104]:
dv.remove_field('TOBT')

In [91]:
# r_{M,t} 市场组合每日收益(沪深300)
index = dv.get_ts('close').index
hs300 = ds.index_daily(['000300.SH'], index[0], index[-1], 'close')[0]
hs300.index = index
hs300.rename(columns={'close':'hs300'}, inplace=True)
hs300_pct = hs300.pct_change()
#hs300_pct.head()

In [92]:
# r_{i,t} 股票的每日收益
stocks_pct = dv.get_ts('close_adj').pct_change()
#stocks_pct.head()

In [98]:
# TORate
dv.add_field('turnover_ratio')
dv.add_field('turnover')
dv.add_field('free_turnover_ratio')
TORate = dv.get_ts('turnover_ratio')
TORate.head()

Field name [turnover_ratio] already exists.
Query data - query...
NOTE: price adjust method is [post adjust]
当前请求daily...
{'adjust_mode': None, 'fields': 'turnover,trade_date,symbol,trade_status'}
下载进度647/1388.
下载进度1294/1388.
    At fields 600005.SH, 600656.SH, 600832.SH, 601299.SH, 601313.SH
Query data - daily fields prepared.
Query data - query...
当前请求query_lb_dailyindicator...
{'fields': 'trade_date,symbol,free_turnover_ratio'}
下载进度647/1388.
下载进度1294/1388.
    At fields 600005.SH, 600656.SH, 600832.SH, 601299.SH, 601313.SH
Query data - daily fields prepared.


symbol,600000.SH,600004.SH,600005.SH,600006.SH,600007.SH,600008.SH,600009.SH,600010.SH,600011.SH,600012.SH,...,603987.SH,603988.SH,603989.SH,603990.SH,603991.SH,603993.SH,603996.SH,603997.SH,603998.SH,603999.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20150105,3.4415,3.4102,NaN,3.2685,1.0827,3.2159,3.3973,4.0052,1.4154,1.1343,...,NaN,6.3350,NaN,NaN,NaN,1.5500,NaN,NaN,11.4802,NaN
20150106,3.4289,2.0757,NaN,3.2635,0.5184,5.6394,3.1586,2.4029,0.9386,2.7095,...,NaN,7.1060,NaN,NaN,NaN,2.8701,NaN,NaN,18.8795,NaN
20150107,2.5848,1.1608,NaN,1.6336,0.4350,4.3040,2.3238,2.6747,0.7350,1.2585,...,NaN,5.6675,NaN,NaN,NaN,2.1118,NaN,NaN,14.1890,NaN
20150108,2.2156,1.0011,NaN,1.2612,0.2536,3.6269,1.8118,1.8171,0.6568,0.8652,...,NaN,4.4082,NaN,NaN,NaN,1.2564,NaN,NaN,9.4517,NaN
20150109,3.2970,1.3386,NaN,0.9386,0.3023,2.3644,2.0552,2.0058,0.6457,0.7344,...,NaN,4.1192,NaN,NaN,NaN,1.6043,NaN,NaN,6.8702,NaN


In [102]:
dv.get_ts('turnover_ratio')

symbol,600000.SH,600004.SH,600005.SH,600006.SH,600007.SH,600008.SH,600009.SH,600010.SH,600011.SH,600012.SH,...,603987.SH,603988.SH,603989.SH,603990.SH,603991.SH,603993.SH,603996.SH,603997.SH,603998.SH,603999.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
20150105,3.4415,3.4102,NaN,3.2685,1.0827,3.2159,3.3973,4.0052,1.4154,1.1343,...,NaN,6.3350,NaN,NaN,NaN,1.5500,NaN,NaN,11.4802,NaN
20150106,3.4289,2.0757,NaN,3.2635,0.5184,5.6394,3.1586,2.4029,0.9386,2.7095,...,NaN,7.1060,NaN,NaN,NaN,2.8701,NaN,NaN,18.8795,NaN
20150107,2.5848,1.1608,NaN,1.6336,0.4350,4.3040,2.3238,2.6747,0.7350,1.2585,...,NaN,5.6675,NaN,NaN,NaN,2.1118,NaN,NaN,14.1890,NaN
20150108,2.2156,1.0011,NaN,1.2612,0.2536,3.6269,1.8118,1.8171,0.6568,0.8652,...,NaN,4.4082,NaN,NaN,NaN,1.2564,NaN,NaN,9.4517,NaN
20150109,3.2970,1.3386,NaN,0.9386,0.3023,2.3644,2.0552,2.0058,0.6457,0.7344,...,NaN,4.1192,NaN,NaN,NaN,1.6043,NaN,NaN,6.8702,NaN
20150112,2.4646,0.9912,NaN,0.7986,0.3762,2.0735,1.6564,1.6138,0.6749,0.6102,...,NaN,4.8801,NaN,NaN,NaN,1.0480,NaN,NaN,7.8831,NaN
20150113,1.1955,0.8091,NaN,0.5622,0.2296,1.3076,0.9125,1.2250,0.3829,0.4185,...,NaN,4.1269,NaN,NaN,NaN,0.9834,NaN,NaN,6.9743,NaN
20150114,2.0708,0.5983,NaN,0.9463,0.1425,1.6478,1.1003,1.8372,0.4588,0.4130,...,NaN,2.5975,NaN,NaN,NaN,1.1059,NaN,NaN,5.2435,NaN
20150115,2.2102,0.8398,NaN,0.4958,0.2081,1.7318,1.0995,1.3529,0.4739,0.4760,...,NaN,4.1363,NaN,NaN,NaN,0.9536,NaN,NaN,5.5343,NaN


# CashRateOfSalesLatest

经营活动产生的现金流量净额与营业收入之比（Cash rate of sales）。计算方法：经营活动产生的现金流量净额与营业收入之比=经营活动产生的现金流量净额（TTM）/营业收入（TTM）。属于质量类因子。数据更新时间：T日21点30分。

P42

In [127]:
def CashRateOfSalesLatest(): 
    # CashRateOfSalesLatest = NOCF/OR
    # NOCF -- 经营活动产生的现金呢流量净额(TTM)
    # OR   -- 营业收入(TTM)

    # 营业收入(TTM)
    dv.add_field('oper_rev')
    # 经营活动产生的现金流量净额
    dv.add_field('net_cash_flows_oper_act')
    
    CashRateOfSalesLatest = dv.add_formula("CashRateOfSalesLatest","net_cash_flows_oper_act/oper_rev", is_quarterly=False, add_data=True)
    return CashRateOfSalesLatest

# OperCashFlowPS

#每股指标

In [156]:
def OperCashFlowPS():
    # OperCashFlowPS = NOCF/TS
    # NOCF -- 经营活动产生的现金呢流量净额(TTM)
    # TS   -- 总股本
    
    # 经营活动产生的现金流量净额
    dv.add_field('net_cash_flows_oper_act')
    # 总股本
    dv.add_field('capital_stk')
    OperCashFlowPS = dv.add_formula("OperCashFlowPS", "net_cash_flows_oper_act/capital_stk", is_quarterly=False, add_data=True)
    return OperCashFlowPS

In [159]:
if 'MA10RegressCoeff12' not in dv.fields:
    print('Ignore')
else:
    print('Add')


Ignore
